In [ ]:
#Basic Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud   #
import re
#Sklean Libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pickle
#Natural Language Processing Libraries
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords      #a group of words that don’t add any value to the sentence
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report

df_train = pd.read_csv('/content/drive/MyDrive/fakeNewsData/train.csv')
df_train.shape
df_train.head()


In [ ]:
df_train = df_train.fillna('')
df_train['merge'] = df_train['author']+' '+ df_train['title']


In [ ]:
wnlm = WordNetLemmatizer()
def text_preprocessor(text):
  transformed_text = re.sub('[^a-zA-Z]',' ',str(text))        #Remove all characters which are not alphabets
  transformed_text = transformed_text.lower()                 #Convert all alphabets to lower case to avoid adding unnecessary complexities
  transformed_text = transformed_text.split()                 #Split the obtained sentence into words
  transformed_text = [wnlm.lemmatize(word) for word in transformed_text if word not in set(stopwords.words('english'))]
  transformed_text = ' '.join(transformed_text)
  return transformed_text

df_train['merge'] = df_train['merge'].apply(text_preprocessor)
print(df_train['merge'])


In [ ]:

x = df_train['merge'].values
y = df_train['label'].values

tfd = TfidfVectorizer()
tfd.fit(x)

x = tfd.transform(x)



In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify=y, random_state=2)

In [ ]:
SVCClf = SVC(kernel = 'linear',gamma = 'scale', shrinking = False,)
SVCClf.fit(x_train, y_train)


In [ ]:
x_train_prediction = SVCClf.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

In [ ]:
x_test_prediction = SVCClf.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)
print('Testing Data accuracy : ', test_data_accuracy)

In [ ]:
filename = 'newsPredictor.sav'
pickle.dump(SVCClf, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open('/content/drive/MyDrive/newsPredictor.sav', 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)